In [1]:
%load_ext autoreload
%autoreload 2

import requests
import pandas as pd
import adnymics.munging_pipeline as munging
import adnymics.write_to_disk as write_to_disk
import os
import pysftp
 
start_date='16-03-2020'
end_date='23-03-2020'
start_time = 'T00:00:00'
end_time = 'T00:00:00'
path = 'd:\\Projects\\datasets\\adnymics'

# Make API call to Chronos
url = ('https://chronos.adnymics.com/v1/feedback/' 
    + str(pd.to_datetime(start_date, format='%d-%m-%Y').strftime('%Y-%m-%d'))
    + str(start_time)
    +'/'
    + str(pd.to_datetime(end_date, format='%d-%m-%Y').strftime('%Y-%m-%d'))
    + str(end_time))
# Store response
response = requests.get(url, auth=('OrsayInternational', '24WTWm7Z'))
# Store resulting JSON
data = response.json()
# Store as dataframe
df = pd.DataFrame(data['brochures'])

# Munge data
df_clean = (df
.pipe(munging.extract_coupons)
.pipe(munging.extract_item_number)
.pipe(munging.drop_cols)
.pipe(munging.split_data)
)

# Write to Disk
# os.chdir('J:\\Group\\CRM\\14_aCRM\\04_ AD HOC\\WEEKLY FIGURES\\Adnymix\\exported_dat')
os.chdir('D:\\')
filename = (str(pd.to_datetime(start_date, format='%d-%m-%Y').strftime('%Y-%m-%d'))
         + str(start_time).replace(':', '-')
         +'__'
         + str(pd.to_datetime(end_date, format='%d-%m-%Y').strftime('%Y-%m-%d'))
         + str(end_time).replace(':', '-')
         + '.xlsx')
#write_to_disk.write_to_disk(df_clean, output.rstrip('.json').rstrip(os.getcwd()+'\\'))
write_to_disk.write_to_disk(df_clean, filename)

print('')
print('DONE!')

# Export to FTP server

myHostname = "ftp.s7.exacttarget.com"
myUsername = "7275356"
myPassword = "orsay2017#FTP"

remotepath = '/Export/Adnymics/'+ str(filename)
localpath = 'd:/'
print(localpath)
cnopts = pysftp.CnOpts()
cnopts.hostkeys = None

with pysftp.Connection(host=myHostname, username=myUsername, password=myPassword, cnopts=cnopts) as sftp:
    print("Connection succesfully established ... ")
    with pysftp.cd(localpath):
        sftp.put(localpath=localpath+filename, remotepath=remotepath)        
sftp.close()